In [3]:
from src.utils.spark import get_spark

# Crear sesión de Spark
spark = get_spark("LendingClubMetrics")

# Cargar el CSV comprimido en GZIP
df = spark.read.csv("Loan_status_2007-2020Q3.gzip", header=True, inferSchema=True)

In [8]:
# Mostrar esquema de la base de datos
df.printSchema()

# Número de columnas
num_cols = len(df.columns)
print(f"Número de columnas: {num_cols}")

# Número de registros
num_rows = df.count()
print(f"Número de registros: {num_rows}")

root
 |-- _c0: integer (nullable = true)
 |-- id: string (nullable = true)
 |-- loan_amnt: integer (nullable = true)
 |-- funded_amnt: integer (nullable = true)
 |-- funded_amnt_inv: double (nullable = true)
 |-- term: string (nullable = true)
 |-- int_rate: string (nullable = true)
 |-- installment: double (nullable = true)
 |-- grade: string (nullable = true)
 |-- sub_grade: string (nullable = true)
 |-- emp_title: string (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_inc: string (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- issue_d: string (nullable = true)
 |-- loan_status: string (nullable = true)
 |-- pymnt_plan: string (nullable = true)
 |-- url: string (nullable = true)
 |-- purpose: string (nullable = true)
 |-- title: string (nullable = true)
 |-- zip_code: string (nullable = true)
 |-- addr_state: string (nullable = true)
 |-- dti: string (nullable = true)
 |-- delinq_2yrs: dou

In [7]:
from pyspark.sql import functions as F

# Calcular el total de registros
total_records = df.count()

# Calcular el número de valores nulos por columna
missing_data = df.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in df.columns])

# Convertir los resultados a un formato más manejable
missing_data = missing_data.toPandas().transpose()

# Renombrar columnas para facilitar la lectura
missing_data.columns = ['missing_values']
missing_data['percentage_missing'] = (missing_data['missing_values'] / total_records) * 100

# Calcular el porcentaje total de datos faltantes en el DataFrame
total_missing_percentage = (missing_data['missing_values'].sum() / (total_records * len(df.columns))) * 100

# Mostrar las 5 columnas con más datos faltantes
top_5_missing_columns = missing_data.sort_values(by='missing_values', ascending=False).head(5)

# Mostrar el porcentaje total de datos faltantes
print(f"Porcentaje total de datos faltantes: {total_missing_percentage:.2f}%")
print("Top 5 columnas con más datos faltantes:")
print(top_5_missing_columns[['missing_values', 'percentage_missing']])

Porcentaje total de datos faltantes: 25.97%
Top 5 columnas con más datos faltantes:
                      missing_values  percentage_missing
hardship_loan_status         2782082           95.097886
hardship_reason              2781861           95.090332
hardship_status              2781858           95.090229
hardship_dpd                 2781856           95.090161
hardship_length              2781855           95.090127


In [16]:
from pyspark.sql import functions as F

df = df.withColumn("dti", F.col("dti").cast("float"))

# Filtrar los valores no numéricos (nulos) de la columna dti
df_cleaned = df.na.drop(subset=["dti"])

# Calcular el monto promedio de los préstamos
avg_loan_amount = df.select(F.avg("loan_amnt")).collect()[0][0]

# Calcular la mediana del monto de los préstamos
median_loan_amount = df.approxQuantile("loan_amnt", [0.5], 0.05)[0]

# Calcular el rango de tasas de interés (mínimo y máximo)
min_int_rate, max_int_rate = df.select(F.min("int_rate"), F.max("int_rate")).collect()[0]

# Calcular el promedio de tasa de interés, ignorando los valores nulos
avg_int_rate = df.na.drop(subset=["int_rate"]).select(F.avg("int_rate")).collect()[0][0]

# Calcular el promedio de ingresos anuales
avg_annual_income = df.select(F.avg("annual_inc")).collect()[0][0]

# Calcular el rango de deuda/ingreso (mínimo y máximo)
min_dti, max_dti = df_cleaned.select(F.min("dti"), F.max("dti")).collect()[0]

# Calcular el promedio de deuda/ingreso
avg_dti = df_cleaned.select(F.avg("dti")).collect()[0][0]

# Mostrar los resultados
print(f"Monto promedio de los préstamos: ${avg_loan_amount:,.2f}")
print(f"Mediana del monto de los préstamos: ${median_loan_amount:,.2f}")
print(f"Rango de tasas de interés: {min_int_rate}% a {max_int_rate}%")
print(f"Promedio de tasa de interés: {avg_int_rate:.2f}%" if avg_int_rate is not None else "Promedio de tasa de interés: No disponible")
print(f"Promedio de ingresos anuales: ${avg_annual_income:,.2f}")
print(f"Rango de deuda/ingreso: {min_dti}% a {max_dti}%")
print(f"Promedio de deuda/ingreso: {avg_dti:.2f}%")

Monto promedio de los préstamos: $15,358.78
Mediana del monto de los préstamos: $12,800.00
Rango de tasas de interés:   5.31%% a  30.99%%
Promedio de tasa de interés: No disponible
Promedio de ingresos anuales: $79,937.29
Rango de deuda/ingreso: -1.0% a 999.0%
Promedio de deuda/ingreso: 19.30%


In [21]:
from pyspark.sql import functions as F
from pyspark.sql.types import NumericType
from pyspark.sql import SparkSession

# 1. Identificación de columnas numéricas
numeric_columns = [field.name for field in df.schema.fields if isinstance(field.dataType, NumericType)]

# 2. Calcular la variabilidad (desviación estándar) y detectar valores atípicos
outlier_counts = {}

for col in numeric_columns:
    # Calcular percentiles (25%, 50%, 75%) para detectar outliers
    quantiles = df.approxQuantile(col, [0.25, 0.5, 0.75], 0.05)
    
    # Calcular la desviación estándar
    stddev = df.select(F.stddev(col)).collect()[0][0]
    
    # Calcular el IQR
    IQR = quantiles[2] - quantiles[0]
    
    # Detectar valores atípicos (outliers)
    lower_bound = quantiles[0] - 1.5 * IQR
    upper_bound = quantiles[2] + 1.5 * IQR
    outliers = df.filter((df[col] < lower_bound) | (df[col] > upper_bound))
    
    # Contar los outliers
    outlier_count = outliers.count()
    
    # Guardar la variabilidad (desviación estándar) y el conteo de outliers
    outlier_counts[col] = {
        "stddev": stddev,
        "outlier_count": outlier_count,
        "percentage_outliers": (outlier_count / df.count()) * 100
    }

# 3. Seleccionar las 10 columnas con más outliers
sorted_columns = sorted(outlier_counts.items(), key=lambda x: x[1]["outlier_count"], reverse=True)[:10]

# 4. Mostrar las 10 columnas con mayor variabilidad y outliers
print("Top 10 columnas con mayor variabilidad y valores atípicos:")
for col, stats in sorted_columns:
    print(f"Columna: {col}")
    print(f"  Desviación estándar: {stats['stddev']:.2f}")
    print(f"  Número de outliers: {stats['outlier_count']}")
    print(f"  Porcentaje de outliers: {stats['percentage_outliers']:.2f}%\n")

Top 10 columnas con mayor variabilidad y valores atípicos:
Columna: num_accts_ever_120_pd
  Desviación estándar: 1.36
  Número de outliers: 633225
  Porcentaje de outliers: 21.65%

Columna: delinq_2yrs
  Desviación estándar: 0.84
  Número de outliers: 518500
  Porcentaje de outliers: 17.72%

Columna: pub_rec
  Desviación estándar: 0.52
  Número de outliers: 426174
  Porcentaje de outliers: 14.57%

Columna: tot_coll_amt
  Desviación estándar: 7496.93
  Número de outliers: 420152
  Porcentaje de outliers: 14.36%

Columna: out_prncp_inv
  Desviación estándar: 7484.51
  Número de outliers: 380168
  Porcentaje de outliers: 13.00%

Columna: pub_rec_bankruptcies
  Desviación estándar: 0.36
  Número de outliers: 340161
  Porcentaje de outliers: 11.63%

Columna: recoveries
  Desviación estándar: 845.91
  Número de outliers: 265248
  Porcentaje de outliers: 9.07%

Columna: mths_since_recent_bc
  Desviación estándar: 32.62
  Número de outliers: 257250
  Porcentaje de outliers: 8.79%

Columna: col

In [ ]:
spark.stop()
